<a href="https://colab.research.google.com/github/KhyatiMahendru/MusicGenreClassification/blob/master/MusicGenreClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Genre Classification: Rock v/s Hip-Hop

In [0]:
#importing required dependencies and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

In [5]:
uploaded = files.upload()

Saving echonest-metrics.json to echonest-metrics.json
Saving fma-rock-vs-hiphop.csv to fma-rock-vs-hiphop (1).csv


In [6]:

tracks = pd.read_csv(io.BytesIO(uploaded['fma-rock-vs-hiphop.csv']))
echonest_metrics = pd.read_json(io.BytesIO(uploaded['echonest-metrics.json']))
echo_tracks = pd.merge(echonest_metrics, tracks[["track_id", "genre_top"]], on = "track_id")
echo_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4802 entries, 0 to 4801
Data columns (total 10 columns):
track_id            4802 non-null int64
acousticness        4802 non-null float64
danceability        4802 non-null float64
energy              4802 non-null float64
instrumentalness    4802 non-null float64
liveness            4802 non-null float64
speechiness         4802 non-null float64
tempo               4802 non-null float64
valence             4802 non-null float64
genre_top           4802 non-null object
dtypes: float64(8), int64(1), object(1)
memory usage: 412.7+ KB


In [7]:
corr_metrics = echo_tracks.corr()
corr_metrics.style.background_gradient()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,1,-0.372282,0.0494541,0.140703,-0.275623,0.0482307,-0.0269951,-0.0253918,0.0100698
acousticness,-0.372282,1,-0.0289537,-0.281619,0.19478,-0.0199914,0.072204,-0.0263097,-0.0138406
danceability,0.0494541,-0.0289537,1,-0.242032,-0.255217,-0.106584,0.276206,-0.242089,0.473165
energy,0.140703,-0.281619,-0.242032,1,0.0282377,0.113331,-0.109983,0.195227,0.0386027
instrumentalness,-0.275623,0.19478,-0.255217,0.0282377,1,-0.0910218,-0.366762,0.022215,-0.219967
liveness,0.0482307,-0.0199914,-0.106584,0.113331,-0.0910218,1,0.0411725,0.00273169,-0.0450931
speechiness,-0.0269951,0.072204,0.276206,-0.109983,-0.366762,0.0411725,1,0.00824055,0.149894
tempo,-0.0253918,-0.0263097,-0.242089,0.195227,0.022215,0.00273169,0.00824055,1,0.0522212
valence,0.0100698,-0.0138406,0.473165,0.0386027,-0.219967,-0.0450931,0.149894,0.0522212,1


In [0]:

X = echo_tracks.drop(['track_id','genre_top'], axis = 1)
Y = echo_tracks.loc[:, 'genre_top']
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.33, random_state = 33, stratify = Y)

In [0]:

scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

In [24]:

clf1 = LogisticRegression(random_state = 0, max_iter = 800)
clf1.fit(trainX, trainY)
pred1 = clf1.predict(testX)
print(accuracy_score(testY, pred1))

0.9015772870662461


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [25]:

clf2 = DecisionTreeClassifier()
clf2.fit(trainX, trainY)
pred2 = clf2.predict(testX)
print(accuracy_score(testY, pred2))


0.8782334384858044


In [26]:

clf5 = KNeighborsClassifier(n_neighbors = 10)
clf5.fit(trainX, trainY)
pred5 = clf5.predict(testX)
print(accuracy_score(testY, pred5))

0.9141955835962146


In [27]:

clf6 = GaussianNB()
clf6.fit(trainX, trainY)
pred6 = clf6.predict(testX)
print(accuracy_score(testY, pred6))

0.8958990536277602


In [28]:

clf3 = SVC(gamma='scale', decision_function_shape='ovo')
clf3.fit(trainX, trainY)
pred3 = clf3.predict(testX)
print(accuracy_score(testY, pred3))

0.9211356466876972


In [0]:

np.random.seed(7)

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(trainY)
encoded_Y = encoder.transform(trainY)
encoded_test_Y = encoder.transform(testY)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_test_y = np_utils.to_categorical(encoded_test_Y)

In [0]:
model = Sequential()
model.add(Dense(256, input_shape=(8,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

# model.add(Dense(300))
# model.add(Activation('sigmoid'))
# model.add(Dropout(0.2))

# model.add(Dense(256))
# model.add(Activation('sigmoid'))
# model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(trainX, dummy_y, epochs=800, batch_size = 32, validation_data=(testX, dummy_test_y))

Instructions for updating:
Use tf.cast instead.
Train on 3217 samples, validate on 1585 samples
Epoch 1/800
3217/3217 [==============================] - 1s 227us/step - loss: 0.3750 - acc: 0.8517 - val_loss: 0.2524 - val_acc: 0.9066
Epoch 2/800
3217/3217 [==============================] - 0s 120us/step - loss: 0.2846 - acc: 0.8878 - val_loss: 0.2286 - val_acc: 0.9123
Epoch 3/800
3217/3217 [==============================] - 0s 120us/step - loss: 0.2650 - acc: 0.8956 - val_loss: 0.2279 - val_acc: 0.9123
Epoch 4/800
3217/3217 [==============================] - 0s 118us/step - loss: 0.2410 - acc: 0.9077 - val_loss: 0.2116 - val_acc: 0.9180
Epoch 5/800
3217/3217 [==============================] - 0s 116us/step - loss: 0.2470 - acc: 0.9036 - val_loss: 0.2048 - val_acc: 0.9218
Epoch 6/800
3217/3217 [==============================] - 0s 116us/step - loss: 0.2372 - acc: 0.9123 - val_loss: 0.2051 - val_acc: 0.9167
Epoch 7/800
3217/3217 [==============================] - 0s 117us/step - loss: 0.2

In [37]:
! pip install spotipy

  Stored in directory: /root/.cache/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy


In [0]:
import spotipy